In [1]:
import filepath




In [2]:
import torch
import numpy as np
import sympy as sp
from abc import ABC
from typing import List, Dict, Tuple
from panda3d.core import VBase3
from panda3d.core import Vec3
from panda3d.core import Quat
from panda3d.core import invert

from panda3d_game.game_object import GameObject, PhysicsGameObject
from panda3d.core import TransformState
# 
from util.geometry import batch_transform
from art.basic import geom_frm_faces, create_cylinder_node, create_cylinder, create_sphere_node
from panda3d.core import GeomPrimitive
from panda3d.core import (
    Geom,
    GeomNode,
    GeomTriangles,
    GeomVertexData,
    GeomVertexFormat,
    GeomVertexWriter,
    GeomEnums, Vec3, LPoint3f
)    
from panda3d.core import GeomNode, NodePath, GeomVertexReader, BitMask32
from art.basic import uv_curve_surface, uv_curve_surface_lambda
from panda3d.bullet import (
    BulletWorld, BulletRigidBodyNode, BulletTriangleMesh, 
    BulletTriangleMeshShape, BulletDebugNode,
      BulletHingeConstraint, BulletGenericConstraint,
BulletConvexHullShape
)
# class RigidNodePath(NodePath):
#     def setPos
from panda3d.core import DirectionalLight, Vec3, PointLight
from sympy.physics.units import (
    kilometer, meter,centimeter,
    gram, kilogram, tonne,
    newton, second
)
from demos.ball_room import MassedBall,tmoon
from art.basic import create_sphere_node
import sys
import filepath
import pdb
from demos.starlight_qt import *
import torch
from qpanda3d import QControl # FIXME
from panda3d_game.app import ControlShowBase, UniversalGravitySpace
from panda3d.bullet import BulletRigidBodyNode
from panda3d.bullet import BulletSphereShape, BulletCylinderShape

In [3]:
class TestScene(
    StarScene, 
    UniversalGravitySpace, 
    QControl
    # ControlShowBase
):
    def __init__(self):
        StarScene.__init__(self)
        unit = {
            "mass" : tonne,
            "length" : 10*meter,
            "time": 1 * second,
            # "force" : sp.Number(1e3) * newton
        }
        G_game = 0.001
        UniversalGravitySpace.__init__(self, unit, G_game)
        QControl.__init__(self)
        shape1 = BulletCylinderShape(1,1,1)
        self.geom1 = create_cylinder_node(height=0.5,height_bot=-0.5,radius=1,lon_res=8,name='1')
        shape2 = BulletSphereShape(1)
        self.geom2 = create_sphere_node(lat_res=8,lon_res=8,name='2')
        body1 = BulletRigidBodyNode('1')
        body1.addShape(shape1)
        body1.setMass(1)
        self.body1 = body1
        self.body1_np = NodePath(body1)
        self.geom1_np = NodePath(self.geom1)
        self.geom1_np.reparentTo(self.body1_np)
        self.geom1_np.setColor((0.5,0.2,0.2,1))
        body2 = BulletRigidBodyNode('2')
        body2.addShape(shape2)
        body2.setMass(1)
        self.body2 = body2
        self.body2_np = NodePath(body2)
        self.geom2_np = NodePath(self.geom2)
        self.geom2_np.reparentTo(self.body2_np)
        self.geom2_np.setColor((0.2,0.5,0.2,1))
        self.body2_np.setPos((1,1,2))
        self.body1_np.reparentTo(self.rdr_scene)
        # self.body2_np.reparentTo(self.rdr_scene)
        self.body2_np.reparentTo(self.body1_np)
        self.body1.setLinearSleepThreshold(0)
        self.body1.setAngularSleepThreshold(0)
        self.body2.setLinearSleepThreshold(0)
        self.body2.setAngularSleepThreshold(0)
        point_light = PointLight('light')
    
        
        point_light.setColor((1, 1, 1, 1))  # 设置光源颜色 (白色)
        point_light.setShadowCaster(True)
        light_np = self.render.attachNewNode(point_light)
        light_np.setPos(2, 2, 2)
        self.render.setLight(light_np)  # 将光源应用到场景
        
        # point_light.set_sc
        dire_light =DirectionalLight('light')
        dire_light.setDirection((-1,-1,-1))
        dire_light.set_color((1, 1, 1, 1.0)) 
        dire_light.setShadowCaster(True)
        light_dr = render.attachNewNode(dire_light)
        # light_dr.setPos(1, 2, 6)  # 设置光源位置
        
        self.render.setShaderAuto()
        self.render.setLight(light_dr)
        self.startQt()
        # self.ship.toBulletWorld(self.bullet_world)
        # self.shell.toBulletWorld(self.bullet_world)
        # self.planet1.toBulletWorld(self.bullet_world)
        # self.taskMgr.add(self.update_torque_)
        self.accept("x", self.add_torque, [10,0,0])
        self.accept("y", self.add_torque, [0,10,0])
        self.accept("c", self.add_torque, [0,0,10])


        debug_node = BulletDebugNode('Debug')
        debug_node.showWireframe(True)
        debug_node.showConstraints(True)
        debug_node.showBoundingBoxes(False)
        debug_node.showNormals(True)
        debug_np = self.rdr_scene.attach_new_node(debug_node)
        self.bullet_world.set_debug_node(debug_node)
        debug_np.show()
        self.mass = body1
        # self.bullet_world.attach_rigid_body(body1)
        # self.bullet_world.attach_rigid_body(body2)
        # self.bullet_world.remove_rigid_body(body1)
        # self.bullet_world.remove_rigid_body(body2)
        self.body1_np.setPos((10,0,0))
        self.bullet_world.attach_rigid_body(body1)
        self.bullet_world.attach_rigid_body(body2)
        # self.body2_np.setPos((1,1,2))
        # self.body1.set_linear_velocity(Vec3(5, 0, 0))
        # print(self.body1_np.getPos(),self.body2_np.getPos())
        self.objects = [self.body1_np,self.geom1_np, self.body2_np,self.geom2_np]

        # self.gravitational_bodies = [self.body1_np, self.body1_np]
    # def update_torque_(self, task):
    #     # self.ship.flywheel.apply_torque_on_axis()
    #     self.ship.shell.rigid_node.applyTorque((1,0,0))
    #     return task.cont

    def add_torque(self, x=0, y=0, z=0):
        
        # print("add torque")
        # This happens in world space
        # mass.apply_torque_impulse(VBase3(x, y, z))
        # print("add torque")
        # self.planet1.rigid_node.apply_torque_impulse(VBase3(x,y,z))
        self.mass.apply_torque(VBase3(x,y,z))
        # self.mass.apply_torque_impulse(VBase3(x,y,z))
        # self.ship.shell.rigid_node.apply_torque_impulse(VBase3(x,y,z))

class TestConsole(PhyscRoomConsole):
    def __init__(self, showbase:PhysicsShowBase) -> None:
        PhyscRoomConsole.__init__(self, showbase)
        self.command_dict.update({
            "getp":(self.print_pos,"print position for all objs")
        })

    def print_pos(self):
        self.log("{}".format([
            body.getPos(self.showbase.rdr_scene) for body in self.objects
        ]), "output")

class TestView(RawQtGUI):
    def get_game(self):
        return TestScene()

    def get_console(self):
        # return PhyscRoomConsole(showbase=self.panda3d)
        return TestConsole(showbase=self.panda3d)
    

In [ ]:
if __name__ == '__main__':
    # torch.set_printoptions(precision=16, sci_mode=False)
    import sys
    app = QApplication(sys.argv)
    window = TestView()
    window.show()
    sys.exit(app.exec_())

Known pipe types:
  glxGraphicsPipe
(all display modules loaded.)
:audio(error): Couldn't open default OpenAL device
:audio(error): OpenALAudioManager: No open device or context
:audio(error):   OpenALAudioManager is not valid, will use NullAudioManager
:audio(error): Couldn't open default OpenAL device
:audio(error): OpenALAudioManager: No open device or context
:audio(error):   OpenALAudioManager is not valid, will use NullAudioManager
/media/ywatcher/ExtDisk1/LDisk/Packages/game_env/env/lib/python3.11/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /home/conda/feedstock_root/build_artifacts/libtorch_1711322746054/work/aten/src/ATen/native/TensorShape.cpp:3526.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
